# Aplicacion Codebar con Flask

In [ ]:
# librerias

from flask import Flask, render_template, Response

import cv2
from pyzbar.pyzbar import decode
from PIL import Image
import pandas as pd
import ast
import pymongo
import requests
import matplotlib.pyplot as plt
%matplotlib inline


import warnings
warnings.simplefilter('ignore')

In [ ]:
def captura():
    cam=cv2.VideoCapture(0)
    while 1:
        ret, frame=cam.read()
        if ret==False: break
        
        key=cv2.waitKey(1)
        
        if key%256==27: break # tecla ESC
        
        elif key%256==32: # SPACE
            img_name='capturas/codigo.png'
            cv2.imwrite(img_name, frame)
            break
            
        frame=cv2.imencode('.jpg', frame)[1].tobytes()
        yield (b'--frame\r\n'b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
        
        
    cam.release()
    cv2.destroyAllWindows()


In [ ]:
def decodifica():
    codigo=decode(Image.open('capturas/codigo.png'))[0][0]
    return codigo

In [ ]:
def elemento(df, codigo):
    
    elem=df.loc[df.ean==str(ast.literal_eval(codigo.decode('utf8')))[0:-1]]
    
    display(elem)

    res=requests.get(elem.URL[elem.index[0]], stream=True)
    img=Image.open(res.raw)

    plt.imshow(img);
    
    return elem

In [ ]:
def busqueda_similares(df, elem, campo):
    
    productos=df.loc[df[campo]==elem[campo][elem.index[0]]]
    
    return productos

In [ ]:
app=Flask(__name__)

In [ ]:
df=''

@app.before_first_request  
def startup():
    global df
    
    df=pd.read_csv('data.csv')

In [ ]:
@app.route('/video')
def video():

    return Response(captura(), mimetype='multipart/x-mixed-replace; boundary=frame')

In [ ]:
@app.route('/', methods=['POST', 'GET'])
def main():
    return render_template('index.html')

In [ ]:
if __name__=='__main__':
    app.run(debug=False)